# Simple Recommender

Now that I've completed some preliminary EDA and engineering, I'll be using this notebook to build the basic recommender that uses the 'taste' field provided by the total wine website, the price and the spirit style. I want to start with a simple one and then later build out a more complex version later on that uses notes of users who left reviews.  

In the end of this notebook I look at a few ways to make recommendations for myself and my coworker Nick based on inputs of 3 difference scotches that we each have enjoyed in the past.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read in the model data, this will be used for the similarty comparison
data = pd.read_csv('./data/simple_rec__model_data.csv')
data.rename(columns = {'Unnamed: 0': 'Name'}, inplace = True)
data.set_index('Name', inplace = True)

In [3]:
# Read in the result data, this will be the output after a similarity has been reached
info = pd.read_csv('./data/simple_rec_info_data.csv')
info.rename(columns = {'title': 'Name'}, inplace = True)
info.set_index('Name', inplace = True)
info.head(3)

,price,taste,brand,description,spirit_style,state
Name,,,,,,
A.D. Rattray Aberfeldy 18 Yr,99.99,None,A.D. Rattray,"Highlands, Scotland- Pleasant aromas of caram...",Single Malt,Highland
A.D. Rattray Auchroisk 22 Yr,139.99,None,A.D. Rattray,"Speyside, Scotland - This Auchroisk is quite ...",Single Malt,NaN
A.D. Rattray Aultmore 16 Year,97.99,None,A.D. Rattray,"Speyside, Scotland- Aroma of honey, ripe frui...",Single Malt,NaN


In [28]:
info[info['taste'] != 'None'].head()

,price,taste,brand,description,spirit_style,state
Name,,,,,,
A.D. Rattray Ben Nevis 17Yr Single Cask Barrel Select,99.99,Delicate . Apple . Grass . Vanilla . Complex,A.D. Rattray,NaN,Single Malt,Highland
A.D. Rattray Girvan 1964 50 Yr,899.99,Rich . Orange . Chocolate . Nut . Long,A.D. Rattray,"Lowlands, Scotland - This 50 year old express...",Single Malt,Lowland
A.D. Rattray Glen Elgin 1995 Single Barrel Select,119.99,Medium . Vanilla . Spice . Long,A.D. Rattray,"Speyside, Scotland- Bright gold color, with a...",Single Malt,NaN
A.D. Rattray Stronachie 12yr,49.99,Peat . Nut,A.D. Rattray,"Highlands, Scotland- Slightly syrupy, malty, ...",Single Malt,Highland
Aberfeldy 12 yo,32.99,Rich . Spice . Honey . Long,Aberfeldy,"Highlands, Scotland- Reserved as the heart of...",Single Malt,Highland


In [30]:
info[info.index.str.contains('glenfiddich',case = False, regex = True)]

,price,taste,brand,description,spirit_style,state
Name,,,,,,
Glenfiddich 12 Yr,26.99,Rich . Peat . Oak . Balanced,Glenfiddich,"Speyside, Scotland- The pioneer of the Single...",Single Malt,Speyside
Glenfiddich 14 Yr Bourbon Barrel Reserve,43.99,None,Glenfiddich,"Speyside, Scotland - Strong oak influence on ...",Single Malt,Speyside
Glenfiddich 15 Yr,52.99,Rich . Sherry . Spice . Long,Glenfiddich,"Speyside, Scotland-Richly layered whisky is m...",Single Malt,Speyside
Glenfiddich 18 Yr,89.99,Intense . Apple . Smoke . Long,Glenfiddich,"Speyside, Scotland- An exceptionally smooth S...",Single Malt,Speyside
Glenfiddich 1963 Replica Release,89.99,None,Glenfiddich,"Speyside, Scotland - Glenfiddich's Malt Maste...",Single Malt,NaN
Glenfiddich 1973 Reserve,5899.99,Rich . Spice . Nut . Long,Glenfiddich,"Speyside, Scotland- The first vintage reserve...",Single Malt,Speyside
Glenfiddich 1978 Vintage Cask,3999.99,Rich . Fruit . Vanilla . Oak . Long,Glenfiddich,"Speyside, Scotland - The nose is balanced wit...",Single Malt,Speyside
Glenfiddich 21 Yr,179.99,Intense . Toffee . Spice . Long,Glenfiddich,"Speyside, Scotland- Intense and sweet, floral...",Single Malt,Speyside
Glenfiddich 26 Yr,449.99,Intense . Vanilla . Oak . Spice . Complex,Glenfiddich,"Speyside, Scotland - A rare and aged single m...",Single Malt,NaN


In [ ]:
['Glenfiddich 12 Yr', 'Glenfiddich 14 Yr']

# Creating the Recommendation Function.  
- I would like to create a function that can take in up to 3 scotches and provide an output of 5 scotches.

In [5]:
# My scotches I want a recommendation for.
my_scotches = ['Laphroaig 10 Yr','Laphroaig Quarter Cask','Lagavulin 16 Yr']

In [6]:
from sklearn.metrics import pairwise_distances

**`make_sim_matrix`** will make a cosign similarity matrix based on the the input of how important price is.

In [7]:
# Price Importance
# 0 = Not Important
# 1 = Somewhat important
# 2 = Important

def make_sim_matrix(sco_data, price_importance = 0):
    df = sco_data.copy()
    # create sim matrix
    if price_importance == 0:
        df.drop('price_scaled', axis =1, inplace = True)
    elif price_importance ==1:
        df['price_scaled'] = df['price_scaled'] * 1.5
    elif price_importance ==2:
        df['price_scaled'] = df['price_scaled'] * 2.0
    else:
        print("Invalid Price Importance")
        return
    
    sim_matrix = pairwise_distances(df,df, metric = 'euclidean')

    # convert to a readable DF
    sim_df = pd.DataFrame(sim_matrix, index = df.index, columns = df.index)
    
    return sim_df

In [8]:
# Price Importance
# 0 = Not Important
# 1 = Somewhat important
# 2 = Important

def recommend(scotch_list, price_importance = 0, sco_df = data, info_df = info ):
    # call make sim function
    df = make_sim_matrix(sco_data = data, price_importance = price_importance)
    input_scotch = df.loc[scotch_list].T
    input_scotch['total'] = input_scotch.sum(axis = 1)
    input_scotch.drop(scotch_list, axis = 0, inplace = True)
    
    recommendations = input_scotch.sort_values(by= 'total',).head().index
    
    return info_df.loc[recommendations]


### My Recommendations

In [9]:
# Money is no concern
my_scotches = ['Laphroaig 10 Yr','Laphroaig Quarter Cask','Lagavulin 16 Yr']
recommend(my_scotches, 0)

,price,taste,brand,description,spirit_style,state
Name,,,,,,
Talisker 18 Yr,149.99,Intense . Smoke . Peat . Balanced,Talisker,"Isle of Skye, Scotland - From the island of S...",Single Malt,Islay & Islands
Dalwhinnie Distillers Edition,76.99,Rich . Peat . Malt . Long,Dalwhinnie,"Highlands, Scotland- Fresh sherry nose, peaty...",Single Malt,Highland
Bowmore 25 Yr,349.99,Rich . Peat . Balanced,Bowmore,"Wine Enthusiast -Islay, Scotland- ""Sniffings ...",Single Malt,NaN
Glenfiddich 12 Yr,26.99,Rich . Peat . Oak . Balanced,Glenfiddich,"Speyside, Scotland- The pioneer of the Single...",Single Malt,Speyside
Shieldaig Islay Single Malt,16.99,Rich . Peat . Smoke . Balanced,Shieldaig,"Islay, Scotland- A classic Islay Malt with po...",Single Malt,Islay & Islands


In [10]:
# Money is some concern
recommend(my_scotches, 1)

,price,taste,brand,description,spirit_style,state
Name,,,,,,
Talisker 18 Yr,149.99,Intense . Smoke . Peat . Balanced,Talisker,"Isle of Skye, Scotland - From the island of S...",Single Malt,Islay & Islands
Dalwhinnie Distillers Edition,76.99,Rich . Peat . Malt . Long,Dalwhinnie,"Highlands, Scotland- Fresh sherry nose, peaty...",Single Malt,Highland
Glenfiddich 12 Yr,26.99,Rich . Peat . Oak . Balanced,Glenfiddich,"Speyside, Scotland- The pioneer of the Single...",Single Malt,Speyside
Shieldaig Islay Single Malt,16.99,Rich . Peat . Smoke . Balanced,Shieldaig,"Islay, Scotland- A classic Islay Malt with po...",Single Malt,Islay & Islands
Amrut Peated Single Malt,59.99,Peat . Fruit . Balanced,Amrut,"India- Aromas of sweet cured bacon, smoke, ma...",Single Malt,NaN


In [11]:
# Money is a good concern
recommend(my_scotches, 2)

,price,taste,brand,description,spirit_style,state
Name,,,,,,
Talisker 18 Yr,149.99,Intense . Smoke . Peat . Balanced,Talisker,"Isle of Skye, Scotland - From the island of S...",Single Malt,Islay & Islands
Dalwhinnie Distillers Edition,76.99,Rich . Peat . Malt . Long,Dalwhinnie,"Highlands, Scotland- Fresh sherry nose, peaty...",Single Malt,Highland
Glenfiddich 12 Yr,26.99,Rich . Peat . Oak . Balanced,Glenfiddich,"Speyside, Scotland- The pioneer of the Single...",Single Malt,Speyside
Shieldaig Islay Single Malt,16.99,Rich . Peat . Smoke . Balanced,Shieldaig,"Islay, Scotland- A classic Islay Malt with po...",Single Malt,Islay & Islands
Berrys' Glen Moray 9yr,59.99,. Balanced,Berrys',"Speyside, Scotland - This very fine single ma...",Single Malt,Speyside


_I'll try the "Glenfiddich 12 yr" and the "Shieldaig Islay Single Malt"._

---

### Nicks Recommendations

In [12]:
# If nick didn't care about money
nicks_scotches = ['Aberfeldy 12 yo', 'Compass Box The Peat Monster','Yamazaki Single 12Y Malt Whisky']
recommend(nicks_scotches, 0)

,price,taste,brand,description,spirit_style,state
Name,,,,,,
Kavalan Whisky Single Barrel Select,179.99,Delicate . Dried Fruit . Honey . Balanced,Kavalan,"Taiwan - Matured in top-quality, Spanish olor...",Single Malt,NaN
Balvenie 21 Yr Port Wood,199.99,Rich . Honey . Spice . Long,Balvenie,"Speyside, Scotland- To create this remarkable...",Single Malt,Speyside
Glenrothes 1998,84.99,Delicate . Honey . Spice . Balanced,Glenrothes,"Speyside, Scotland- Cereal, honey and sweet, ...",Single Malt,Speyside
Exclusive Casks Bladnoch 25 Yr,169.99,Rich . Fruit . Honey . Balanced,Exclusive Casks,"Lowland, Scotland - A Lowland single malt of ...",Single Malt,Lowland
Glencadam 15 Yr,94.99,Rich . Honey . Smoke . Balanced,Glencadam,"Highlands, Scotland- Malt and citrus fruits o...",Single Malt,Highland


In [13]:
# if nick cared a bit about money
recommend(nicks_scotches, 1)

,price,taste,brand,description,spirit_style,state
Name,,,,,,
Kavalan Whisky Single Barrel Select,179.99,Delicate . Dried Fruit . Honey . Balanced,Kavalan,"Taiwan - Matured in top-quality, Spanish olor...",Single Malt,NaN
Glenrothes 1998,84.99,Delicate . Honey . Spice . Balanced,Glenrothes,"Speyside, Scotland- Cereal, honey and sweet, ...",Single Malt,Speyside
Glencadam 15 Yr,94.99,Rich . Honey . Smoke . Balanced,Glencadam,"Highlands, Scotland- Malt and citrus fruits o...",Single Malt,Highland
Exclusive Casks Bladnoch 25 Yr,169.99,Rich . Fruit . Honey . Balanced,Exclusive Casks,"Lowland, Scotland - A Lowland single malt of ...",Single Malt,Lowland
Berrys' Glen Moray 9yr,59.99,. Balanced,Berrys',"Speyside, Scotland - This very fine single ma...",Single Malt,Speyside


In [14]:
# if nick cared quite a bit about money
recommend(nicks_scotches, 2)

,price,taste,brand,description,spirit_style,state
Name,,,,,,
Glenrothes 1998,84.99,Delicate . Honey . Spice . Balanced,Glenrothes,"Speyside, Scotland- Cereal, honey and sweet, ...",Single Malt,Speyside
Glencadam 15 Yr,94.99,Rich . Honey . Smoke . Balanced,Glencadam,"Highlands, Scotland- Malt and citrus fruits o...",Single Malt,Highland
Kavalan Whisky Single Barrel Select,179.99,Delicate . Dried Fruit . Honey . Balanced,Kavalan,"Taiwan - Matured in top-quality, Spanish olor...",Single Malt,NaN
Berrys' Glen Moray 9yr,59.99,. Balanced,Berrys',"Speyside, Scotland - This very fine single ma...",Single Malt,Speyside
AnCnoc 12 Yr,36.99,Rich . Honey . Fruit . Long,AnCnoc,"Highlands, Scotland- Soft and very aromatic n...",Single Malt,Highland


_I think the "Balvenie 12 Yr Double Wood" and "AnCnoc 12 Yr" would be decent recommendations. (Though that "Lagavulin 16 Yr" is friggn goooooooood)_

---

**These recommendations were made off of minimal taste notes provided by the website.  The next step is to build a more complete model that utilizes the descriptions left by the reviewers. **

# Further Recommendations

Nick actuall had a third scotch, the "Yamazaki Single 12Y Malt Whisky" that I could not initially find when filtering my df for whatever reason. So, im going to start adding that in and see if there are any changes.

In [15]:
nicks_scotches_2 = ['Aberfeldy 12 yo', 'Compass Box The Peat Monster','Yamazaki Single 12Y Malt Whisky']
recommend(nicks_scotches_2, 0)

,price,taste,brand,description,spirit_style,state
Name,,,,,,
Kavalan Whisky Single Barrel Select,179.99,Delicate . Dried Fruit . Honey . Balanced,Kavalan,"Taiwan - Matured in top-quality, Spanish olor...",Single Malt,NaN
Balvenie 21 Yr Port Wood,199.99,Rich . Honey . Spice . Long,Balvenie,"Speyside, Scotland- To create this remarkable...",Single Malt,Speyside
Glenrothes 1998,84.99,Delicate . Honey . Spice . Balanced,Glenrothes,"Speyside, Scotland- Cereal, honey and sweet, ...",Single Malt,Speyside
Exclusive Casks Bladnoch 25 Yr,169.99,Rich . Fruit . Honey . Balanced,Exclusive Casks,"Lowland, Scotland - A Lowland single malt of ...",Single Malt,Lowland
Glencadam 15 Yr,94.99,Rich . Honey . Smoke . Balanced,Glencadam,"Highlands, Scotland- Malt and citrus fruits o...",Single Malt,Highland


In [16]:
nicks_scotches_2 = ['Aberfeldy 12 yo', 'Compass Box The Peat Monster','Yamazaki Single 12Y Malt Whisky']
recommend(nicks_scotches_2, 1)

,price,taste,brand,description,spirit_style,state
Name,,,,,,
Kavalan Whisky Single Barrel Select,179.99,Delicate . Dried Fruit . Honey . Balanced,Kavalan,"Taiwan - Matured in top-quality, Spanish olor...",Single Malt,NaN
Glenrothes 1998,84.99,Delicate . Honey . Spice . Balanced,Glenrothes,"Speyside, Scotland- Cereal, honey and sweet, ...",Single Malt,Speyside
Glencadam 15 Yr,94.99,Rich . Honey . Smoke . Balanced,Glencadam,"Highlands, Scotland- Malt and citrus fruits o...",Single Malt,Highland
Exclusive Casks Bladnoch 25 Yr,169.99,Rich . Fruit . Honey . Balanced,Exclusive Casks,"Lowland, Scotland - A Lowland single malt of ...",Single Malt,Lowland
Berrys' Glen Moray 9yr,59.99,. Balanced,Berrys',"Speyside, Scotland - This very fine single ma...",Single Malt,Speyside


In [17]:
nicks_scotches_2 = ['Aberfeldy 12 yo', 'Compass Box The Peat Monster','Yamazaki Single 12Y Malt Whisky']
recommend(nicks_scotches_2, 2)

,price,taste,brand,description,spirit_style,state
Name,,,,,,
Glenrothes 1998,84.99,Delicate . Honey . Spice . Balanced,Glenrothes,"Speyside, Scotland- Cereal, honey and sweet, ...",Single Malt,Speyside
Glencadam 15 Yr,94.99,Rich . Honey . Smoke . Balanced,Glencadam,"Highlands, Scotland- Malt and citrus fruits o...",Single Malt,Highland
Kavalan Whisky Single Barrel Select,179.99,Delicate . Dried Fruit . Honey . Balanced,Kavalan,"Taiwan - Matured in top-quality, Spanish olor...",Single Malt,NaN
Berrys' Glen Moray 9yr,59.99,. Balanced,Berrys',"Speyside, Scotland - This very fine single ma...",Single Malt,Speyside
AnCnoc 12 Yr,36.99,Rich . Honey . Fruit . Long,AnCnoc,"Highlands, Scotland- Soft and very aromatic n...",Single Malt,Highland


I want to make a not of the "Berrys' Glen Moray 9yr", this has one flavor note "Balanced", which means it recorded a whopping 0 for all but one note.  This is going to cause issues because it has less opportunity for it to extend its difference.  Lets use the below toy example to clarify this issue.

|     x   | Balanced | Peat | Honey | Smoke | Cherry | 
| ------- | :------: | :--: | :---: | :---: | :----: | 
|Scotch 1 |    1     |   0  |   0   |   0   |    0   |
|Scotch 2 |    1     |   0  |   0   |   0   |    1   |
|Scotch 3 |    1     |   0  |   0   |   1   |    1   |
|Scotch 4 |    1     |   1  |   0   |   0   |    1   |

- Scotch 1 only has 1 recorded taste note "Balanced",
- Scotch 2 has 2 recorded taste notes "Balanced" and "Cherry".
- Scotch 3 has 3 recorded taste notes "Balanced", "Smoke" and "Cherry".
- Scotch 4 has 3 recorded taste notes "Balanced", "Peat", and "Cherry".


Scotch 1 and Scotch 2 have a total distance of 1 between them. {(1 - 1) + (0 - 0) + (0 - 0) + (0 - 0) + (0 - 1)}
- Scotch 1 and Scotch 2 only have 1 similarity between them, but they still have a minimized distance.

Scotch 3 and Scotch 4 have a total distance of 2 between them. {(1 - 1) + (0 - 1) + (0 - 0) + (1 - 0) + (1 - 1)}
- Even though Scotch 3 and Scotch 4 have more tastes in common than Scotch 1 and Scotch 2 do, their distance is still greater.  

This is the problem with "Berrys' Glen Moray 9yr" appearing as a recommendation.

##  Single value based recommendations.

In [18]:
recommend(['Compass Box The Peat Monster'],0)

,price,taste,brand,description,spirit_style,state
Name,,,,,,
Talisker 18 Yr,149.99,Intense . Smoke . Peat . Balanced,Talisker,"Isle of Skye, Scotland - From the island of S...",Single Malt,Islay & Islands
Lagavulin 16 Yr,79.99,Intense . Smoke . Peat . Balanced,Lagavulin,"Islay, Scotland- One of the great classics fr...",Single Malt,Islay & Islands
Ardbeg 10yr,59.99,Intense . Smoke . Peat . Long,Ardbeg,"Islay, Scotland- Ardbeg 10 year old Single Ma...",Single Malt,Islay & Islands
Berrys' Benrinnes 16yr,109.99,Intense . Honey . Peat . Long,Berrys',"Speyside, Scotland - This deeply scented whis...",Single Malt,Speyside
Ardbeg Uigeadail,75.99,Intense . Peat . Smoke . Long,Ardbeg,"Islay, Scotland- A marriage of Ardbeg from bo...",Single Malt,Islay & Islands


In [19]:
recommend(['Compass Box The Peat Monster'],1)

,price,taste,brand,description,spirit_style,state
Name,,,,,,
Lagavulin 16 Yr,79.99,Intense . Smoke . Peat . Balanced,Lagavulin,"Islay, Scotland- One of the great classics fr...",Single Malt,Islay & Islands
Talisker 18 Yr,149.99,Intense . Smoke . Peat . Balanced,Talisker,"Isle of Skye, Scotland - From the island of S...",Single Malt,Islay & Islands
Tomintoul Peaty Malt,52.99,Intense . Peat . Smoke . Long,Tomintoul,"Speyside, Scotland- The distillery known for ...",Single Malt,Speyside
Ardbeg 10yr,59.99,Intense . Smoke . Peat . Long,Ardbeg,"Islay, Scotland- Ardbeg 10 year old Single Ma...",Single Malt,Islay & Islands
Nikka Coffey Malt Whisky,59.99,Medium . Peat . Smoke . Balanced,Nikka,Japan- This spirit is the result of a unique ...,Single Malt,NaN


In [20]:
recommend(['Compass Box The Peat Monster'],2)

,price,taste,brand,description,spirit_style,state
Name,,,,,,
Lagavulin 16 Yr,79.99,Intense . Smoke . Peat . Balanced,Lagavulin,"Islay, Scotland- One of the great classics fr...",Single Malt,Islay & Islands
Talisker 18 Yr,149.99,Intense . Smoke . Peat . Balanced,Talisker,"Isle of Skye, Scotland - From the island of S...",Single Malt,Islay & Islands
Tomintoul Peaty Malt,52.99,Intense . Peat . Smoke . Long,Tomintoul,"Speyside, Scotland- The distillery known for ...",Single Malt,Speyside
Ardbeg 10yr,59.99,Intense . Smoke . Peat . Long,Ardbeg,"Islay, Scotland- Ardbeg 10 year old Single Ma...",Single Malt,Islay & Islands
Nikka Coffey Malt Whisky,59.99,Medium . Peat . Smoke . Balanced,Nikka,Japan- This spirit is the result of a unique ...,Single Malt,NaN


Based on these results, I'd probably recommend the [Tomintoul Peaty Malt](https://www.totalwine.com/spirits/scotch/single-malt/tomintoul-peaty-malt/p/104773750?s=1108&igrules=true), [Ardbeg 10yr](https://www.totalwine.com/spirits/scotch/single-malt/ardbeg-10yr/p/95385750?s=2301&igrules=true) or [Nikka Coffey Malt Whisky](https://www.totalwine.com/spirits/scotch/single-malt/nikka-coffey-malt-whisky/p/173800750?s=2301&igrules=true) for Nick.

In [21]:
recommend(['Aberfeldy 12 yo'],0)

,price,taste,brand,description,spirit_style,state
Name,,,,,,
Balvenie 21 Yr Port Wood,199.99,Rich . Honey . Spice . Long,Balvenie,"Speyside, Scotland- To create this remarkable...",Single Malt,Speyside
Johnnie Walker Blue Cities,239.99,Rich . Smoke . Honey . Spice . Long,Johnnie Walker,Scotland- This Johnnie Walker Blue Cities edi...,NaN,NaN
Dalmore 12 Yr,54.99,Rich . Spice . Maple . Long,Dalmore,"Highlands, Scotland- With its intricate aroma...",Single Malt,Highland
Hamilton's Speyside Single Malt Scotch Whisky,21.99,Rich . Spice . Fruit . Long,Hamiltons,"Speyside, Scotland - Single Malt Scotch Whisk...",Single Malt,Speyside
Balblair Barrel Select,179.99,Rich . Spice . Oak . Long,Balblair,"Scotland- The taste is initially sweet, yet s...",Single Malt,Highland


In [22]:
recommend(['Aberfeldy 12 yo'],1)

,price,taste,brand,description,spirit_style,state
Name,,,,,,
Balvenie 21 Yr Port Wood,199.99,Rich . Honey . Spice . Long,Balvenie,"Speyside, Scotland- To create this remarkable...",Single Malt,Speyside
AnCnoc 12 Yr,36.99,Rich . Honey . Fruit . Long,AnCnoc,"Highlands, Scotland- Soft and very aromatic n...",Single Malt,Highland
Balvenie 12 Yr Double Wood,37.49,Rich . Honey . Vanilla . Long,Balvenie,"Speyside, Scotland- Nose is sweet fruit and O...",Single Malt,Speyside
Hamilton's Speyside Single Malt Scotch Whisky,21.99,Rich . Spice . Fruit . Long,Hamiltons,"Speyside, Scotland - Single Malt Scotch Whisk...",Single Malt,Speyside
Hamilton's Highland Single Malt Scotch Whisky,21.99,Rich . Spice . Oak . Long,Hamiltons,"Highland, Scotland - Single Malt Scotch Whisk...",Single Malt,Highland


In [23]:
recommend(['Aberfeldy 12 yo'],2)

,price,taste,brand,description,spirit_style,state
Name,,,,,,
Balvenie 21 Yr Port Wood,199.99,Rich . Honey . Spice . Long,Balvenie,"Speyside, Scotland- To create this remarkable...",Single Malt,Speyside
AnCnoc 12 Yr,36.99,Rich . Honey . Fruit . Long,AnCnoc,"Highlands, Scotland- Soft and very aromatic n...",Single Malt,Highland
Balvenie 12 Yr Double Wood,37.49,Rich . Honey . Vanilla . Long,Balvenie,"Speyside, Scotland- Nose is sweet fruit and O...",Single Malt,Speyside
Hamilton's Speyside Single Malt Scotch Whisky,21.99,Rich . Spice . Fruit . Long,Hamiltons,"Speyside, Scotland - Single Malt Scotch Whisk...",Single Malt,Speyside
Hamilton's Highland Single Malt Scotch Whisky,21.99,Rich . Spice . Oak . Long,Hamiltons,"Highland, Scotland - Single Malt Scotch Whisk...",Single Malt,Highland


Based on these results, I'd probably recommend any of the ones that are not the Balvenie 21 yr for Nick.

#### Final Recommendation : Nick
If I were to use these to recommend 1 scotch to Nick it would be [AnCnoc 12 Yr](https://www.totalwine.com/spirits/scotch/single-malt/ancnoc-12-yr/p/110323750?s=2301&igrules=true)

----

### Single Value Recommendations for me.
- With a moderate emphasis on price.

In [24]:
recommend(['Laphroaig 10 Yr'],2)

,price,taste,brand,description,spirit_style,state
Name,,,,,,
Dalwhinnie Distillers Edition,76.99,Rich . Peat . Malt . Long,Dalwhinnie,"Highlands, Scotland- Fresh sherry nose, peaty...",Single Malt,Highland
Bowmore 12 Yr,43.99,Rich . Peat . Salt . Long,Bowmore,"Wine Enthusiast -Islay, Scotland- ""The unabas...",Single Malt,Islay & Islands
Ledaig 10 Yr,49.99,Rich . Tar . Peat . Long,Ledaig,"Islay, Scotland- Sweet, ripe barley, balanced...",Single Malt,Islay & Islands
Berrys' Ardmore 8yr,59.99,Rich . Peat . Fruit . Long,Berrys',"Highlands, Scotland - The nose gives some but...",Single Malt,Highland
Isle of Jura 16 Yr,74.99,Rich . Spice . Peat . Long,Isle of Jura,"Isle of Jura, Scotland- Full and rich. Soft p...",Single Malt,Islay & Islands


In [25]:
recommend(['Laphroaig Quarter Cask'],1)

,price,taste,brand,description,spirit_style,state
Name,,,,,,
Deanston 12 Yr,49.99,Delicate . Nut . Peat . Balanced,Deanston,"Highlands, Scotland- Dry and nutty, with ligh...",Single Malt,Highland
Jura 10 Yr,43.99,Delicate . Salt . Peat . Balanced,Jura,"Isle of Jura, Scotland- The 10 year old varia...",Single Malt,Islay & Islands
Tomatin Single Malt 12 Yr,32.99,Delicate . Peat . Wood . Balanced,Tomatin,"Highlands, Scotland- Elegant and airy, with a...",Single Malt,Highland
Glen Ness 12 Yr Single Malt Scotch Whisky,27.99,Delicate . Spice . Oak . Balanced,Glen Ness,"Highland, Scotland- Clean and creamy with a b...",Single Malt,Highland
Glenfiddich 12 Yr,26.99,Rich . Peat . Oak . Balanced,Glenfiddich,"Speyside, Scotland- The pioneer of the Single...",Single Malt,Speyside


In [26]:
recommend(['Lagavulin 16 Yr'],1)

,price,taste,brand,description,spirit_style,state
Name,,,,,,
Talisker 18 Yr,149.99,Intense . Smoke . Peat . Balanced,Talisker,"Isle of Skye, Scotland - From the island of S...",Single Malt,Islay & Islands
Compass Box The Peat Monster,49.99,Intense . Peat . Smoke . Honey . Balanced,Compass Box,"Wine Enthusiast -Islay, Scotland- ""An intense...",NaN,NaN
Ardbeg Uigeadail,75.99,Intense . Peat . Smoke . Long,Ardbeg,"Islay, Scotland- A marriage of Ardbeg from bo...",Single Malt,Islay & Islands
BenRiach 15 Yr Sherry Wood,84.99,Intense . Peat . Smoke . Long,BenRiach,"Speyside, Scotland- The nose is huge in peat ...",Single Malt,NaN
Nikka Coffey Malt Whisky,59.99,Medium . Peat . Smoke . Balanced,Nikka,Japan- This spirit is the result of a unique ...,Single Malt,NaN


#### Final Recommendation : Sam
If I were to recommend a single scotch for myself it would be the [Glenfiddich 12 Yr](https://www.totalwine.com/spirits/scotch/single-malt/glenfiddich-12-yr/p/2789750?s=2301&igrules=true) or the [Jura 10 Yr](https://www.totalwine.com/spirits/scotch/single-malt/jura-10-yr/p/1776750?s=2301&igrules=true)

In [32]:
recommend(['Glenfiddich 12 Yr', 'Glenfiddich 15 Yr'],1)

,price,taste,brand,description,spirit_style,state
Name,,,,,,
Grangestone 12Yr Single Malt Scotch Whisky,33.99,Rich . Spice . Oak . Balanced,Grangestone,"Highlands, Scotland - This scotch spends twel...",Single Malt,Highland
McClelland's Highland,27.99,Rich . Spice . Peat . Balanced,McClelland's,"Highlands, Scotland- Specially chosen to high...",Single Malt,Highland
Hamilton's Highland Single Malt Scotch Whisky,21.99,Rich . Spice . Oak . Long,Hamiltons,"Highland, Scotland - Single Malt Scotch Whisk...",Single Malt,Highland
Isle of Jura 16 Yr,74.99,Rich . Spice . Peat . Long,Isle of Jura,"Isle of Jura, Scotland- Full and rich. Soft p...",Single Malt,Islay & Islands
Glenlivet 18 Yr,89.99,Rich . Spice . Oak . Long,Glenlivet,"Speyside, Scotland- The Glenlivet Master Dist...",Single Malt,Speyside


In [33]:
info.loc[['Glenfiddich 12 Yr', 'Glenfiddich 15 Yr']]

,price,taste,brand,description,spirit_style,state
Name,,,,,,
Glenfiddich 12 Yr,26.99,Rich . Peat . Oak . Balanced,Glenfiddich,"Speyside, Scotland- The pioneer of the Single...",Single Malt,Speyside
Glenfiddich 15 Yr,52.99,Rich . Sherry . Spice . Long,Glenfiddich,"Speyside, Scotland-Richly layered whisky is m...",Single Malt,Speyside
